## This code corresponds to Example 3 of the [arXiv preprint](https://arxiv.org/abs/2404.13578)

Extends the hemodynamics problem considered in Example 2 to 3D.

Tested with NGSolve version 6.2.2402

In [ ]:
from netgen.occ import *
from ngsolve import *
from ngsolve.webgui import *
import matplotlib.pyplot as plt
import numpy as np

### Model coefficients and parameters

In [ ]:
t = Parameter(0.0)

dt = 1e-4
    
#mass density
rhoF = 1
muF  = 0.035
lamF = 1e6
    
rhoS = 1.1
    
#Lamé coef. corresponding to C
muS  = 0.575e6
lamS = 1.7e6

#beta = 4e6    
#needed for A = C**{-1}
aS1 = 0.5 / muS
aS2 = lamS / (2.0 * muS * (3 * lamS + 2 * muS))
    
aF1 = 0.5 / muF
aF2 = lamF / (2.0 * muF * (3 * lamF + 2 * muF))

pMax = 1.333e4
tMax = 0.003
pIn = IfPos(tMax - t, 0.5*pMax*( 1 - cos(2*np.pi*t/tMax) )  , 0)

### Geometry and mesh

The default length of the cylinder is set to L=2.

The results presented in the article were obtained using L=5. 
It's worth noting that running the code with this larger value of L 
significantly increases the computational time.

In [ ]:
L = 2

Cylout  = Cylinder ( Pnt(0,0,0), X, r = 0.6, h = L ).bc("gammaN")
Cylin   = Cylinder ( Pnt(0,0,0), X, r = 0.5, h = L ).bc("sigma")

solid = Cylout - Cylin
fluid  = Cylout*Cylin

solid.mat("solid")
fluid.mat("fluid")

solid.faces.Max(X).bc("gammaD")
solid.faces.Min(X).bc("gammaD")

fluid.faces.Max(X).bc("gammaOut")
fluid.faces.Min(X).bc("gammaIn")

tube = Glue( [fluid, solid] )

mesh = Mesh(OCCGeometry(tube).GenerateMesh(maxh=0.25)).Curve(3)
Draw (mesh, clipping=True);

#mesh.GetMaterials()
#mesh.GetBoundaries()


### Finite element spaces

In [ ]:
domain_values_rho = {'fluid': rhoF,  'solid': rhoS}
values_list_rho = [domain_values_rho[mat] for mat in mesh.GetMaterials()]
rho = CoefficientFunction(values_list_rho)

k = 3 #order of the finite element space

S = L2(mesh, order = k)
W = VectorL2(mesh, order = k+1)
What0 = FacetFESpace(mesh, order=k+1, dirichlet="gammaD")
What1 = FacetFESpace(mesh, order=k+1, dirichlet="gammaD|gammaIn|gammaOut")
What2 = FacetFESpace(mesh, order=k+1, dirichlet="gammaD|gammaIn|gammaOut")
fes = FESpace([S, S, S, S, S, S, W, What0, What1, What2])

sigma11, sigma12, sigma13, sigma22, sigma23, sigma33, u, uhat0, uhat1, uhat2 = fes.TrialFunction()
tau11,   tau12,   tau13,   tau22,   tau23,   tau33,   v, vhat0, vhat1, vhat2 = fes.TestFunction()
    
sigma = CoefficientFunction(( sigma11,  sigma12,  sigma13,  sigma12,  sigma22,  sigma23,  sigma13,  sigma23,  sigma33),   dims = (3,3) )
tau   = CoefficientFunction(( tau11,    tau12,    tau13,    tau12,    tau22,    tau23,    tau13,    tau23,    tau33),     dims = (3,3) )

uhat = CoefficientFunction(( uhat0, uhat1, uhat2))
vhat = CoefficientFunction(( vhat0, vhat1, vhat2))

AsigmaS = aS1 * sigma - aS2 * Trace(sigma) *  Id(mesh.dim)
AsigmaF = aF1 * sigma - aF2 * Trace(sigma) *  Id(mesh.dim)

jump_u = u - uhat
jump_v = v - vhat

### Bilinear forms and right-hand side

In [ ]:
n = specialcf.normal(mesh.dim)
h = specialcf.mesh_size
dS = dx(element_boundary=True)

a = BilinearForm(fes, condense=True)
a += (1/dt)*rho*u*v*dx
a += (1/dt)*InnerProduct(AsigmaS, tau)*dx("solid") 
a +=    0.5*InnerProduct(AsigmaF,tau)*dx("fluid")
a +=    0.5*InnerProduct(sigma, grad(v))*dx    - 0.5*InnerProduct(grad(u), tau)*dx
a += -  0.5*InnerProduct( sigma*n, jump_v)*dS  + 0.5*InnerProduct(jump_u, tau*n)*dS
a +=    0.5*((k+1)**2/h)*jump_u*jump_v*dS
a.Assemble()

inv_A = a.mat.Inverse(freedofs=fes.FreeDofs(coupling=True))
    
M = BilinearForm(fes)
M += (1/dt)*rho*u*v*dx
M += (1/dt)*InnerProduct(AsigmaS, tau)*dx("solid")
M += - 0.5*InnerProduct(AsigmaF,tau)*dx("fluid")
M += - 0.5*InnerProduct(sigma, grad(v))*dx   + 0.5*InnerProduct(grad(u), tau)*dx
M +=   0.5*InnerProduct( sigma*n, jump_v)*dS - 0.5*InnerProduct(jump_u, tau*n)*dS
M += - 0.5*((k+1)**2/h)*jump_u*jump_v*dS
M.Assemble()

ft = LinearForm(fes)
#ft += source * v * dx#("fluid") + sourceS * v * dx("solid")
ft += - pIn*( vhat0.Trace()*n[0] + vhat1.Trace()*n[1] + vhat2.Trace()*n[2] )*ds(definedon=mesh.Boundaries("gammaIn"))

### Instantiation of initial conditions

In [ ]:
disp0 = GridFunction(W)
disp0.vec[:] = 0.0

disp = GridFunction(W)
    
u0 = GridFunction(fes)
u0.vec[:] = 0.0

u1 = GridFunction(fes)
    
ft.Assemble()
    
res = u0.vec.CreateVector()
b0  = u0.vec.CreateVector()
b1  = u0.vec.CreateVector()
    
b0.data = ft.vec

t_intermediate = dt # time counter within one block-run

### Time loop

In [ ]:
tend = 12e-3 # end time

while t_intermediate < tend:
    
    t.Set(t_intermediate)
    ft.Assemble()
    b1.data = ft.vec
     
    res.data = M.mat*u0.vec + 0.5*(b0.data + b1.data)

    u1.vec[:] = 0.0 

    res.data = res - a.mat * u1.vec
    res.data += a.harmonic_extension_trans * res
    u1.vec.data += inv_A * res
    u1.vec.data += a.inner_solve * res
    u1.vec.data += a.harmonic_extension * u1.vec
    
       
    disp.vec[:] = 0.0
    disp.vec.data = disp0.vec + 0.5*dt*(u1.components[6].vec + u0.components[6].vec)
        
    u0.vec.data = u1.vec
    disp0.vec.data = disp.vec
    b0.data = b1.data
    
    
    #computing relevant data for the graphical representation
    VH = VectorL2(mesh, order=k+1, definedon = "solid")
    dis = GridFunction(VH, 'displacement')
    dis.Set( disp0 )

    VF = VectorL2(mesh, order=k+1, definedon = "fluid")
    velo = GridFunction(VF, 'flow')
    velo.Set( u0.components[6])

    Mises = sqrt(    0.5*( u0.components[0]   - u0.components[3]  )**2 
               + 0.5*( u0.components[3]   - u0.components[5]  )**2  
               + 0.5*( u0.components[5]  -  u0.components[0]  )**2 
               + 3*u0.components[1]**2 
               + 3*u0.components[4]**2 
               + 3*u0.components[2]**2  )

    pre = -(u0.components[0] + u0.components[3] + u0.components[5])/3

    H = L2(mesh, order=k, definedon = "solid")
    vonMises = GridFunction(H, 'vonMises')
    vonMises.Set(Mises)

    P = L2(mesh, order=k, definedon = "fluid")
    pressure = GridFunction(P, 'pressure')
    pressure.Set(pre)
    
    #Export data for a graphical representation in Paraview at time steps 0.004, 0.008 and 0.012
    if abs(t_intermediate - 0.004) < 1e-5:
        vtk = VTKOutput(ma=mesh, coefs=[dis, vonMises, pressure, velo], names = ["displacement",  "von Mises stress", "pressure", "velocity"], filename="biGtube4", subdivision=3)
        vtk.Do()
    
    if abs(t_intermediate - 0.008) < 1e-5:
        vtk = VTKOutput(ma=mesh, coefs=[dis, vonMises, pressure, velo], names = ["displacement",  "von Mises stress", "pressure", "velocity"], filename="biGtube8", subdivision=3)
        vtk.Do()
        
    if abs(t_intermediate - 0.012) < 1e-5:
        vtk = VTKOutput(ma=mesh, coefs=[dis, vonMises, pressure, velo], names = ["displacement",  "von Mises stress", "pressure", "velocity"], filename="biGtube12", subdivision=3)
        vtk.Do()
    ##########################
    
    print("\r",t_intermediate,end="")
    
    t_intermediate += dt

### Export line-data for figures

In [ ]:
disp_values = []
pressure_values = []

start = 0
end = L
step = 1/20

partition = np.arange(start, end+step, step)

for p in partition:
    disp_values.append(disp0(mesh(p, 0.55, 0))[1])
    pressure_values.append( pressure(mesh(p, 0.0, 0.0)) )


fig, axs = plt.subplots(2)  # Create a figure with 4 subplots

#Plot disp_values
axs[0].plot(partition, disp_values)
axs[0].set_xlabel('interface line')
axs[0].set_ylabel('y-disp')
axs[0].grid(True)

#Plot pressure_values
axs[1].plot(partition, pressure_values)
axs[1].set_xlabel('central line')
axs[1].set_ylabel('pressure')
axs[1].grid(True)

plt.tight_layout()  # Adjust the layout so that plots do not overlap
plt.show()